## Import Libaries

In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, ToPILImage
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
import torch.nn.functional as F

## Exploring the Data

In [ ]:
with tarfile.open('../input/cifar10/cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path = './input')

In [ ]:
print(os.listdir('./input/cifar10'))
classes = os.listdir('./input/cifar10/train')
print(classes)

In [ ]:
cat_files = os.listdir('./input/cifar10/train/cat')
print('No of training example for cats : ', len(cat_files))
print(cat_files[:5])

In [ ]:
automobile_files = os.listdir('./input/cifar10/train/automobile')
print('No of training example for automobiles : ', len(automobile_files))
print(automobile_files[:5])

In [ ]:
cat_files = os.listdir('./input/cifar10/test/cat')
print('No of test example for cats : ', len(cat_files))
print(cat_files[:5])

In [ ]:
data_dir = './input/cifar10'
dataset = ImageFolder(data_dir + '/train', transform = ToTensor())

In [ ]:
img, label = dataset[0]
print(img.shape, label)
img

In [ ]:
dataset

In [ ]:
print(dataset.classes)

In [ ]:
def show_image(img, label):
    print('Label : ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
show_image(dataset[222][0], dataset[222][1])

In [ ]:
show_image(*dataset[1099])

## Training and Validation Datasets

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

In [ ]:
val_size = 5000
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size], generator = torch.manual_seed(random_seed))
len(train_ds), len(val_ds)

In [ ]:
batch_size = 128
train_dl = DataLoader(train_ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_ds, batch_size, shuffle = True, num_workers = 4, pin_memory = True)

In [ ]:
train_dl

In [ ]:
def show_images_batch(d1):
    for images, labels in d1:
        fig, ax = plt.subplots(figsize = (16, 8))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute(1, 2, 0))
        break

In [ ]:
show_images_batch(train_dl)

In [ ]:
show_images_batch(val_dl)

## Convolutional Neural Network

In [ ]:
def apply_kernel(image, kernel):
    ri, ci = image.shape
    rk, ck = kernel.shape
    ro, co = ri-rk+1, ci-ck+1
    output = torch.zeros([ro, co])
    for i in range(ro):
        for j in range(co):
            output[i, j] = torch.sum(image[i:i+rk, j:j+ck] * kernel)
    return output

In [ ]:
sample_image = torch.tensor([
    [4, 4, 3, 1, 0],
    [0, 0, 1, 4, 1],
    [4, 1, 3, 3, 4],
    [2, 0, 0, 3, 3],
    [3, 0, 0, 0, 1]
], dtype = torch.float32)

sample_kernel = torch.tensor([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
], dtype = torch.float32)

apply_kernel(sample_image, sample_kernel)

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim = 1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        return {'val_loss' : loss.detach(), 'val_acc' : acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss' : epoch_loss.item(), 'val_acc' : epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print('Epoch [{}], train_loss {:.4f}, val_loss : {:.4f}, val_acc : {:.4f}'.format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        
            nn.Flatten(),
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model = Cifar10CnnModel()
model

In [ ]:
for images, labels in train_dl:
    print('images.shape : ', images.shape)
    out = model(images)
    print('out.shape : ', out.shape)
    print('out[0]', out[0])
    break

In [ ]:
out.shape

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking = True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

## Training the Model

In [ ]:
@torch.no_grad()

def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    
    for epoch in range(epochs):
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
model

In [ ]:
evaluate(model, val_dl)

In [ ]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 0.001

In [ ]:
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy cs No of epochs')

In [ ]:
plt.figure(figsize = (10, 6))
plot_accuracies(history)

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss Vs. No. of epochs')

In [ ]:
plt.figure(figsize = (10, 6))
plot_losses(history)

## Testing with individual images

In [ ]:
test_dataset = ImageFolder(data_dir+'/test', transform = ToTensor())
test_dataset

In [ ]:
def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds = torch.max(yb, dim = 1)
    return dataset.classes[preds[0].item()]

In [ ]:
img, label = test_dataset[8]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[1220]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[345]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[6153]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[456]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[10]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
img, label = test_dataset[1432]
plt.imshow(img.permute(1, 2, 0))
print('Label : ', dataset.classes[label], ', Predicted : ', predict_image(img, model))

In [ ]:
test_loader = DeviceDataLoader(DataLoader(test_dataset, batch_size), device)
test_result = evaluate(model, test_loader)
test_result